In [3]:
import pandas as pd
import numpy as np

# For Training Dataset

In [4]:
df1=pd.read_csv(r"D:\\Learn\\Project\\CDC25\train(1).csv")

In [5]:
print(df1.shape)
df1.head()

(16209, 21)


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,9117000170,20150505T000000,268643,4,2.25,1810,9240,2.0,0,0,...,7,1810,0,1961,0,98055,47.4362,-122.187,1660,9240
1,6700390210,20140708T000000,245000,3,2.50,1600,2788,2.0,0,0,...,7,1600,0,1992,0,98031,47.4034,-122.187,1720,3605
2,7212660540,20150115T000000,200000,4,2.50,1720,8638,2.0,0,0,...,8,1720,0,1994,0,98003,47.2704,-122.313,1870,7455
3,8562780200,20150427T000000,352499,2,2.25,1240,705,2.0,0,0,...,7,1150,90,2009,0,98027,47.5321,-122.073,1240,750
4,7760400350,20141205T000000,232000,3,2.00,1280,13356,1.0,0,0,...,7,1280,0,1994,0,98042,47.3715,-122.074,1590,8071


In [ ]:
import requests

def fetch_satellite_image(lat, lon, save_path, zoom=17, size=256):
    token = "pk.eyJ1Ijoic2FoaWxoMjIwMyIsImEiOiJjbWp4dDNoeHc2ZDNjM2hxeHBrdjhqbmppIn0.mN5mk0-IbbafXc0UUpxwYw"
    url = (
        f"https://api.mapbox.com/styles/v1/mapbox/satellite-v9/static/"
        f"{lon},{lat},{zoom}/{size}x{size}"
        f"?access_token={token}"
    )

    response = requests.get(url, timeout=10)
    response.raise_for_status()

    with open(save_path, "wb") as f:
        f.write(response.content)


In [16]:
import os
row=df1.iloc[0]
lat1=row["lat"]
long1=row["long"]
base_dir="D:\\Learn\\Project\\CDC25\\data\\images"
file_name="images_0001.png"
save_path1=os.path.join(base_dir,file_name)

In [17]:
fetch_satellite_image(lat1,long1,save_path1)

In [ ]:
import os
import time

MAPBOX_TOKEN = "your_Mapboc_Token"
OUTPUT_DIR = "data/images"
ZOOM = 17
SIZE = 256
SLEEP_SECONDS = 0.2
os.makedirs(OUTPUT_DIR, exist_ok=True)
if not MAPBOX_TOKEN:
    raise RuntimeError("MAPBOX_TOKEN not found. Set environment variable.")

In [32]:
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def create_session():
    retry = Retry(
        total=5,
        backoff_factor=1.5,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["GET"],
        raise_on_status=False,
    )

    adapter = HTTPAdapter(max_retries=retry, pool_connections=10, pool_maxsize=10)

    session = requests.Session()
    session.mount("https://", adapter)
    session.mount("http://", adapter)
    return session


SESSION = create_session()


def fetch_image(lat, lon, image_id):
    url = (
        f"https://api.mapbox.com/styles/v1/mapbox/satellite-v9/static/"
        f"{lon},{lat},{ZOOM}/{SIZE}x{SIZE}"
        f"?access_token={MAPBOX_TOKEN}"
    )

    try:
        response = SESSION.get(url, timeout=30)
        response.raise_for_status()

        path = os.path.join(OUTPUT_DIR, f"{image_id}.png")
        with open(path, "wb") as f:
            f.write(response.content)

    except requests.exceptions.RequestException as e:
        raise RuntimeError(f"Failed for ID {image_id}: {e}")


In [34]:
df_short=df1[:1]
df_short.shape

(1, 21)

In [35]:

for _, row in df_short.iterrows():
    try:
        fetch_image(row["lat"], row["long"], row["id"])
        time.sleep(SLEEP_SECONDS)
    except Exception as e:
        print(f"Failed for ID {row['id']}: {e}")

In [36]:
for _, row in df1.iterrows():
    image_id = row["id"]
    path = os.path.join(OUTPUT_DIR, f"{image_id}.png")

    # Skip if already downloaded
    if os.path.exists(path):
        continue

    try:
        fetch_image(row["lat"], row["long"], image_id)
        time.sleep(SLEEP_SECONDS)

    except Exception as e:
        print(f"Failed for ID {image_id}: {e}")

In [40]:
df1.shape

(16209, 21)

# For Test Dataset

In [42]:
df2=pd.read_csv("D:\\Learn\\Project\\CDC25\\test.csv")

In [43]:
df2.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,2591820310,20141006T000000,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700
1,7974200820,20140821T000000,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283
2,7701450110,20140815T000000,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685
3,9522300010,20150331T000000,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226
4,9510861140,20140714T000000,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050


In [44]:
df2.shape

(5404, 20)

In [46]:
OUTPUT_DIR = "data/test_images"

In [ ]:
for _, row in df2.iterrows():
    image_id = row["id"]
    path = os.path.join(OUTPUT_DIR, f"{image_id}.png")

    if os.path.exists(path):
        continue

    try:
        fetch_image(row["lat"], row["long"], image_id)
        time.sleep(SLEEP_SECONDS)

    except Exception as e:
        print(f"Failed for ID {image_id}: {e}")